In [2]:
import time, os
import argparse, configparser
import numpy as np
from PIL import Image
from numba import jit, prange
from dask import array
from tlz import partition_all
from dask import array as da
import dask.dataframe as df

In [3]:
from dask.distributed import Client, progress
from dask import delayed
client = Client(n_workers=8, threads_per_worker=2, memory_limit='2GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42741,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:34417,Total threads: 2
Dashboard: http://127.0.0.1:36653/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:41553,


In [55]:
from functools import partial
real_min =  -2.5
real_max = +1.5
imag_min = -1.125
imag_max = +1.125
iters_max = 32
width = 1920
height =  1080
name =  'mandelbrot_chunked.jpg'

In [33]:
def mandelbrot_colors_dask_chunked(real_min,real_max,imag_min,imag_max,iters_max,width_f,height_f,chunk
                                   ,block_id=None):
    # image size in pixels
    
    """real_min =  real_min if block_id[0]%2==0 else real_min*-1
    real_max =  real_max if block_id[0]%2==0 else real_max*-1
    imag_min = imag_min if block_id[0]%2==0 else imag_min*-1
    imag_max = imag_max if block_id[0]%2==0 else imag_max*-1"""
    
    """iters_max = 256
    width_f=1920
    height_f=1080"""
    #curent_chnk=int((str(block_id[0])+str(block_id[1])), 2)
    
    width = chunk.shape[0]#int(pixel.shape[0]*curent_chnk if curent_chnk!=0 else  pixel.shape[0])
    height = chunk.shape[1] #int(pixel.shape[1]*curent_chnk if curent_chnk!=0 else pixel.shape[1])
    resuls=np.zeros((height,width,3),dtype=np.uint8)
   
    # pixel size in complex plane

    #start_w=width*curent_chnk#pixel.shape[0]
    #start_h=height*curent_chnk#-pixel.shape[1]

    
    real_step = (real_max - real_min) / width
    imag_step = (imag_max -imag_min) / height

    #client.map(mandel_convergence,height,width,real_min,real_step,imag_step,imag_min,image)
   
    # check convergence of each pixel in the image
    for y in range(0,height):
         for x in range(0,width):

            # a point in a complex plane corrsponding to the pixel (x, y)
            real=real_min+real_step* x#(x*-1 if curent_chnk in range(0,1,1) else x)
            imag =imag_min+imag_step*y#(y*-1 if curent_chnk in range(0,2,2) else y)
                      
            c = complex(real, imag)
           
            # check for convergence
            z = complex(0, 0)
            iters = 0
            
            while abs(z) <= 2 and iters < iters_max:
                z = z*z + c
                iters += 1

            # color pixel in HSV scheme 
                resuls[y][x]=[iters % 255,255,255 * (iters < iters_max)] #[start_w,start_h,curent_chnk]
        # if x==width-1:
            #    break
        #if y==height-1:
         #   break
    return resuls
            

In [93]:
DaskArray=da.zeros((1920,1080,3),chunks=(480,270
                                         ,3))

In [94]:
DaskArray.chunks

((480, 480, 480, 480), (270, 270, 270, 270), (3,))

In [95]:
# parse arguments

#config_file = "default.conf"#args['config']

# main processing
t = time.time()
image=da.map_blocks(mandelbrot_colors_dask_chunked,real_min,real_max,imag_min,
                    imag_max,iters_max, width,height,DaskArray,
                    dtype=np.uint8,chunks=DaskArray.chunks)


image=image.compute()
t = time.time() - t
# save image
#image
Image.fromarray(image, mode='HSV').convert('RGB').save(name)

# printout
print('PAR: size:', (width, height), 'iterations:', iters_max, 'time:', round(t, 3), "s")    

# end main

PAR: size: (1920, 1080) iterations: 32 time: 3.595 s


In [66]:
image.shape

(1080, 1920, 3)

Število skupkov=4*4=16 <br>
velikost =1920 krat 1080 <br>
število iteracij =4 do 256 <br>
število delavcev =8 in niti 2 na delavca <br>


Prenosnik v smislu spreminjanja št. iteracij <br>
št. iteracij | čas(s) <br>
4  | 1.413 <br>
8  | 1.809 <br>
16 | 2.414 <br>
32 | 3.633 <br>
64 | 5.806 <br>
128| 10.014 <br>
256| 18.445 <br>

Prenosik spreminjanje št. skupkov št. iteracij = 32 ostalo enako <br>
št. skupkov | Čas(s) <br>
1  | 20.379 <br>
2  | 10.201 <br>
4  | 5.408 <br>
8  | 3.14 <br>
16 | 3.595